In [1]:
!pip install AmbuView==0.1.3

In [2]:
from AmbuView import data_processing
import pytest
import folium
from shapely.geometry import MultiPolygon
from pkg_resources import resource_filename
import geopandas as gpd
import os
import sys

In [3]:
# testing data_processing.py
# load locations from csv
script_dir = os.path.dirname(sys.argv[0])
locations_csv = resource_filename('AmbuView', 'test_data_torquay_locations.csv')
locations_csv_path = os.path.join(script_dir, locations_csv)
locations_df = data_processing.load_locations_from_csv(locations_csv_path)
print(locations_df)
# should return a Dataframe containing the locations data.

  Location Name   Population  Latitude  Longitude
0        Brixham       17834   50.3951    -3.5000
1       Paignton       67488   50.4351    -3.5642
2        Torquay       52037   50.4619    -3.5253


In [4]:
# testing preprocess_locations_data function
heat_data = data_processing.preprocess_locations_data(locations_df)
# expected output [(50.3951, -3.5, 17834), (50.4351, -3.5642, 67488), (50.4619, -3.5253, 52037)]
print(heat_data)

[(50.3951, -3.5, 17834), (50.4351, -3.5642, 67488), (50.4619, -3.5253, 52037)]


In [5]:
# testing preprocess_locations_data_for_opt
opt_data = data_processing.preprocess_locations_data_for_opt(heat_data)
# expected output [(50.3951, -3.5), (50.4351, -3.5642), (50.4619, -3.5253)]
print(opt_data)

[(50.3951, -3.5), (50.4351, -3.5642), (50.4619, -3.5253)]


In [6]:
torbay_gdf = data_processing.load_devon_boundary("https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/Counties_and_Unitary_Authorities_May_2023_UK_BFC/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson")
torbay_gdf.head() # expected output: geodataframe 

,FID,CTYUA23CD,CTYUA23NM,CTYUA23NMW,BNG_E,BNG_N,LONG,LAT,Shape__Area,Shape__Length,GlobalID,geometry
0,1,E06000001,Hartlepool,,447160,531474,-1.27018,54.6761,9.371703e+07,70995.173776,8b50f3f3-0310-4449-8c87-b3ebef2c225b,"MULTIPOLYGON (((-1.22470 54.62611, -1.22493 54..."
1,2,E06000002,Middlesbrough,,451141,516887,-1.21099,54.5447,5.388156e+07,44479.646370,adcc4b31-117f-4d15-9718-7f8586874528,"MULTIPOLYGON (((-1.27720 54.54784, -1.27721 54..."
2,3,E06000003,Redcar and Cleveland,,464361,519597,-1.00608,54.5675,2.450695e+08,96703.510654,95c18bc7-43c3-498f-bd42-6ccf8b5257f3,"MULTIPOLYGON (((-1.20098 54.57763, -1.20030 54..."
3,4,E06000004,Stockton-on-Tees,,444940,518183,-1.30664,54.5569,2.049329e+08,123404.959102,5199dbc0-068f-4578-b639-b730de7fd3a6,"MULTIPOLYGON (((-1.27211 54.55337, -1.27213 54..."
4,5,E06000005,Darlington,,428029,515648,-1.56835,54.5353,1.974778e+08,107202.299425,688582e5-8a91-4417-9df7-bf345e9972d9,"POLYGON ((-1.63768 54.61714, -1.63767 54.61670..."


In [7]:
torbay_gdf = data_processing.optimize_geodataframe(torbay_gdf, "CTYUA23NM", "Torbay")
print(torbay_gdf)
# expected output: geodataframe containing just the geometry for torbay 

    FID  CTYUA23CD CTYUA23NM CTYUA23NMW   BNG_E  BNG_N     LONG      LAT  \
26   27  E06000027    Torbay             289730  64613 -3.55523  50.4709   

     Shape__Area  Shape__Length                              GlobalID  \
26  6.288674e+07   77909.531551  6d5440d1-1eaf-445b-bd36-bff88837937a   

                                             geometry  
26  POLYGON ((-3.51511 50.51743, -3.50914 50.51658...  


In [8]:
# test get_devon_boundary_polygon
print(data_processing.get_devon_boundary_polygon(torbay_gdf))
# expected output: POLYGON ((-3.5151077200151 50.5174265250422, -3.50913535137322 50.5165786692391...

POLYGON ((-3.5151077200151 50.5174265250422, -3.50913535137322 50.5165786692391, -3.50860600898915 50.515344098181, -3.51160888466258 50.5109440907044, -3.51098803452156 50.5075777775444, -3.51330802451309 50.5061930967733, -3.51248773125484 50.499396957303, -3.51432176260908 50.49762292434, -3.51317635301243 50.496946186573, -3.51531626885635 50.496138597144, -3.5134216634752 50.4948672216611, -3.51458496473578 50.4876751032781, -3.51282479463084 50.4860287245739, -3.51463463386576 50.4851633753026, -3.51479898902573 50.4816889813385, -3.51206161191094 50.4796512030885, -3.50700774305542 50.4794459994469, -3.49742895548181 50.4758757792439, -3.50093504217877 50.4749069805583, -3.5019168989705 50.4727847223387, -3.49058161819083 50.4693774133296, -3.49109633684641 50.4661636142686, -3.48900952237248 50.4638082889147, -3.48059441325885 50.4635180499793, -3.48432042913283 50.4598483393688, -3.48737733096817 50.4607219453014, -3.49129436240642 50.4581237087865, -3.50415137342892 50.459596

In [9]:
f_map = folium.Map(location=[50.38872, -3.51266], zoom_start=11)
torbay_boundry_layer = data_processing.create_devon_boundary_layer(f_map, torbay_gdf)
print(torbay_boundry_layer)
# Showing the torbay_boundry_layer added to the folium map.
torbay_boundry_layer.add_to(f_map)
f_map
# expected output: folium map with the torbay boundries outlined in yellow.

In [10]:
# test read_and_combine_geojson
data_processing.read_and_combine_geojson(["https://raw.githubusercontent.com/martinjc/UK-GeoJSON/master/json/statistical/eng/msoa_by_lad/E07000188.json", 
                        "https://raw.githubusercontent.com/martinjc/UK-GeoJSON/master/json/statistical/eng/msoa_by_lad/E07000189.json"])
# expected output should be a geodataframe with both sedgemoor and south somerset documented. 

,MSOA11CD,MSOA11NM,geometry
0,E02006061,Sedgemoor 001,"POLYGON ((-2.80799 51.32623, -2.80646 51.32615..."
1,E02006062,Sedgemoor 002,"POLYGON ((-3.02218 51.32644, -3.02024 51.32643..."
2,E02006063,Sedgemoor 003,"POLYGON ((-2.88223 51.30466, -2.88143 51.30354..."
3,E02006064,Sedgemoor 004,"POLYGON ((-2.99094 51.25154, -2.99140 51.24978..."
4,E02006065,Sedgemoor 005,"POLYGON ((-2.98213 51.23669, -2.98316 51.23175..."
5,E02006066,Sedgemoor 006,"POLYGON ((-2.99893 51.22237, -2.99691 51.22204..."
6,E02006067,Sedgemoor 007,"MULTIPOLYGON (((-3.02026 51.21581, -3.02305 51..."
7,E02006068,Sedgemoor 008,"POLYGON ((-2.98703 51.14638, -2.98642 51.14336..."
8,E02006069,Sedgemoor 009,"MULTIPOLYGON (((-2.99187 51.13085, -2.99135 51..."
9,E02006070,Sedgemoor 010,"POLYGON ((-2.97230 51.16184, -2.96695 51.15965..."


In [11]:
data_processing.read_and_combine_geojson(["https://raw.githubusercontent.com/martinjc/UK-GeoJSON/master/json/statistical/eng/msoa_by_lad/E06000027.json"])
# checking it still works if only one geojson file is input in the list. 

,MSOA11CD,MSOA11NM,geometry
0,E02003154,Torbay 001,"POLYGON ((-3.53511 50.50515, -3.53317 50.50465..."
1,E02003155,Torbay 002,"POLYGON ((-3.51560 50.51787, -3.51380 50.51655..."
2,E02003156,Torbay 003,"POLYGON ((-3.54279 50.49667, -3.54196 50.49617..."
3,E02003157,Torbay 004,"POLYGON ((-3.53968 50.48532, -3.53896 50.48505..."
4,E02003158,Torbay 005,"POLYGON ((-3.51479 50.48170, -3.51204 50.47969..."
5,E02003159,Torbay 006,"POLYGON ((-3.51969 50.47559, -3.51809 50.47496..."
6,E02003161,Torbay 008,"POLYGON ((-3.53753 50.47608, -3.53726 50.47393..."
7,E02003163,Torbay 010,"POLYGON ((-3.49728 50.47582, -3.50089 50.47493..."
8,E02003164,Torbay 011,"POLYGON ((-3.57902 50.45838, -3.57828 50.45780..."
9,E02003165,Torbay 012,"POLYGON ((-3.57759 50.44687, -3.57741 50.44662..."
